In [46]:
%%local
import os
username = os.environ['JUPYTERHUB_USER']
get_ipython().run_cell_magic('configure', line="-f", cell='{ "name":"%s-final-istaden", "executorMemory":"4G", "executorCores":4, "numExecutors":10, "driverMemory": "4G" }' % username)

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
6956,application_1618324153128_6605,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
6915,application_1618324153128_6543,pyspark,idle,Link,Link,None,
6926,application_1618324153128_6556,pyspark,idle,Link,Link,None,
6927,application_1618324153128_6560,pyspark,idle,Link,Link,None,
6931,application_1618324153128_6566,pyspark,idle,Link,Link,None,
6939,application_1618324153128_6578,pyspark,idle,Link,Link,None,
6940,application_1618324153128_6579,pyspark,idle,Link,Link,None,
6944,application_1618324153128_6584,pyspark,busy,Link,Link,None,
6945,application_1618324153128_6586,pyspark,idle,Link,Link,None,
6946,application_1618324153128_6589,pyspark,idle,Link,Link,None,
6947,application_1618324153128_6591,pyspark,idle,Link,Link,None,


In [47]:
%%send_to_spark -i username -t str -n username

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Successfully passed 'username' as 'username' to Spark kernel

In [48]:
#Loading the data 

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [65]:
df = spark.read.orc('/data/sbb/orc/istdaten')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [66]:
df=df.selectExpr('betriebstag as date',
                 'produkt_id as transport_type',
                 'haltestellen_name as stop_name',
                 'ankunftszeit as arrival_scheduled',
                 'an_prognose as arrival_actual',
                 'durchfahrt_tf as stop_skip',
                 'bpuic as stop_id',
                  'linien_text as line_name',
                  'verkehrsmittel_text as line_type',
                  'linien_id as line_id'
                )

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [67]:
df.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+--------------+--------------------+-----------------+-------------------+---------+-------+---------+---------+----------+
|      date|transport_type|           stop_name|arrival_scheduled|     arrival_actual|stop_skip|stop_id|line_name|line_type|   line_id|
+----------+--------------+--------------------+-----------------+-------------------+---------+-------+---------+---------+----------+
|02.02.2020|           Bus|Winterthur, Schmi...| 02.02.2020 22:55|02.02.2020 22:57:31|    false|8580472|        1|        B|85:882:101|
|02.02.2020|           Bus|Winterthur, Stadt...| 02.02.2020 22:56|02.02.2020 22:58:19|    false|8580521|        1|        B|85:882:101|
|02.02.2020|           Bus| Winterthur, Obertor| 02.02.2020 22:56|02.02.2020 22:59:07|    false|8573656|        1|        B|85:882:101|
|02.02.2020|           Bus|Winterthur, Palms...| 02.02.2020 22:58|02.02.2020 22:59:39|    false|8590982|        1|        B|85:882:101|
|02.02.2020|           Bus|   Winterthur, Swica|

In [68]:
import pyspark.sql.functions as F

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [69]:
#convert day of the trips between
df = df.withColumn('date', F.to_timestamp('date', "dd.MM.yyyy"))
df = df.withColumn('arrival_scheduled', F.to_timestamp('arrival_scheduled', "dd.MM.yyyy HH:mm"))
df = df.withColumn('arrival_actual', F.to_timestamp('arrival_actual', 'dd.MM.yyyy HH:mm:ss'))    

#keep only rows for stops that are not skipped
df=df.filter(df.stop_skip==False)

#keep only rows for stops during the week
df=df.withColumn("day_of_week",F.dayofweek(df.date))
df=df.filter(df.day_of_week.between(2,6))

#hours between 8am and 8pm
min_day_hour,max_day_hour=8,20
df = df.filter(F.hour(F.col('arrival_scheduled')).cast('int').between(min_day_hour, max_day_hour))

#only keep stops within the 15km radius
reachable_stops_path="/user/%s/final/parquet/reachable_stops" %username
stop_id_reachable=spark.read.parquet(reachable_stops_path)
stop_id_reachable=stop_id_reachable.withColumn("stop_id",F.split(stop_id_reachable.stop_id,':')[0]).drop("stop_name")

df=df.join(stop_id_reachable,on="stop_id")

#add hour column
df=df.withColumn("hour",F.hour(F.col("arrival_scheduled")))


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [70]:
df.filter(df.transport_type=="Zug").show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+-------------------+--------------+--------------------+-------------------+-------------------+---------+---------+---------+-------+-----------+----------------+----------------+-------------+--------------+------------+-------------------+----+
|stop_id|               date|transport_type|           stop_name|  arrival_scheduled|     arrival_actual|stop_skip|line_name|line_type|line_id|day_of_week|        stop_lat|        stop_lon|location_type|parent_station|true_stop_id| distance_zurich_HB|hour|
+-------+-------------------+--------------+--------------------+-------------------+-------------------+---------+---------+---------+-------+-----------+----------------+----------------+-------------+--------------+------------+-------------------+----+
|8503097|2018-02-16 00:00:00|           Zug|    Langnau-Gattikon|2018-02-16 08:00:00|2018-02-16 08:00:43|    false|       S4|        S|  24443|          6|47.2868644549589|8.54402039889557|         null|              |     850309

In [71]:
#filter the rows where transport_type is null
df=df.where((F.col("transport_type")=="Tram") | (F.col("transport_type")=="Zug")|(F.col("transport_type")=="Bus"))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [72]:
df.select("transport_type").distinct().show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------+
|transport_type|
+--------------+
|          Tram|
|           Zug|
|           Bus|
+--------------+

In [ ]:
stop_id_reachable.show(10)

In [73]:
#computing actual delays
@F.udf
def relu(x):
    return max(x,0)
df=df.withColumn("delay",relu((F.col("arrival_actual").cast("long")-F.col("arrival_scheduled").cast("long"))))
df = df.where(~(F.col('delay').isNull()))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
#statistics type of transport average delays
transport_avg_delays=df.groupby("transport_type").agg(F.mean('delay'))
transport_avg_delays.show()

In [ ]:
transport_avg_delays=transport_avg_delays.withColumnRenamed("avg(delay)","avg_delay")

In [ ]:
transport_avg_delays.show()

In [ ]:
import matplotlib
matplotlib.use('agg')
import matplotlib.pylab as plt

plt.rcParams['figure.figsize'] = (30,8)
plt.rcParams['font.size'] = 12
plt.style.use('fivethirtyeight')

In [ ]:
import matplotlib.pyplot as plt
# Plot transport average delays

t_avg_delays_plot=transport_avg_delays.toPandas()

plt.bar(x=t_avg_delays_plot.transport_type,height=t_avg_delays_plot.avg_delay)
#transport_avg_delays["avg_delay"].plot.bar
plt.xlabel('average delay distribution by train')
plt.ylabel('average delay (seconds)')
plt.xticks(range(len(t_avg_delays_plot)), t_avg_delays_plot.transport_type)

In [ ]:
%matplot plt

In [ ]:
hour_avg_delays=df.groupby("hour").agg(F.mean('delay'))

hour_avg_delays=hour_avg_delays.withColumnRenamed("avg(delay)","avg_delay")

In [ ]:
# Plot transport average delays
plt.cla()
h_avg_delays_plot=hour_avg_delays.toPandas().sort_values(by="hour").reset_index(drop=True)

plt.bar(x=h_avg_delays_plot.hour, height=h_avg_delays_plot.avg_delay, color=(0.2, 0.4, 0.6, 0.6))
 
plt.xlabel('average delay distribution by hour')
plt.ylabel('average delay (seconds)')
plt.xticks(h_avg_delays_plot.hour)
# Show the graph
%matplot plt

In [ ]:
h_avg_delays_plot

In [57]:
train_types=df.where(F.col("transport_type")=="Zug").groupby("line_name").agg(F.mean('delay'))
#train_types.filter(train_types.line_name[0:2]=="RE").show()
train_types.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

61

In [62]:
df

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[stop_id: string, date: timestamp, transport_type: string, stop_name: string, arrival_scheduled: timestamp, arrival_actual: timestamp, stop_skip: string, line_name: string, line_type: string, line_id: string, day_of_week: int, stop_name: string, stop_lat: double, stop_lon: double, location_type: int, parent_station: string, true_stop_id: string, distance_zurich_HB: double, hour: int, delay: string]

In [75]:
train_types=df.where(F.col("transport_type")=="Zug").groupby("line_name", "hour", "stop_id").agg(F.mean('delay'))
#train_types.filter(train_types.line_name[0:2]=="RE").show()
train_types.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------+----+-------+------------------+
|line_name|hour|stop_id|        avg(delay)|
+---------+----+-------+------------------+
|      S18|   9|8530813|               0.0|
|      S10|  15|8503052|54.899648156646776|
|       S4|  16|8503087|62.882486979166664|
|       S4|  17|8503088| 96.81582446808511|
|      S17|  15|8502268|56.642736486486484|
|       S3|  14|8503020|38.809453471196456|
|       S3|  14|8503305| 39.92926670992862|
|       S3|  20|8503020| 38.55282555282555|
|       S6|   8|8503526| 99.38361929426999|
|       S6|  16|8503007| 99.81791720569211|
|       S6|  19|8503111| 90.99481865284974|
|       S8|   8|8503204|106.08058252427185|
|       S8|  11|8503000|22.001940491591203|
|      S12|  15|8503147| 32.74444444444445|
|      S12|  19|8503147|41.098381466201204|
|      S21|  18|8503008| 79.24966078697422|
|      S15|  18|8503147| 160.9047619047619|
|        S|  17|8503306|            35.375|
|      EXT|  19|8503103|             105.0|
|       S7|  19|8503103|        

In [ ]:
train_types_hour=df.where(F.col("transport_type")=="Zug").groupby("line_type","hour").agg(F.mean('delay'))
train_types_hour.show()

In [ ]:
bus_types=df.where(F.col("transport_type")=="Bus").groupby("line_name").agg(F.mean('delay'))
bus_types.show()

In [ ]:
bus_types_hour=df.where(F.col("transport_type")=="Bus").groupby("line_name", "hour").agg(F.mean('delay'))
bus_types_hour.show()

In [ ]:
tram_types=df.where(F.col("transport_type")=="Tram").groupby("line_name").agg(F.mean('delay'))
tram_types.show()

In [ ]:
tram_types_hour=df.where(F.col("transport_type")=="Tram").groupby("line_name", "hour").agg(F.mean('delay'))
tram_types_hour.show()

In [ ]:
#Renaming
train_types = train_types.withColumnRenamed("avg(delay)","avg_delay")
train_types_hour = train_types_hour.withColumnRenamed("avg(delay)","avg_delay")
bus_types= bus_types.withColumnRenamed("avg(delay)","avg_delay")
bus_types_hour= bus_types_hour.withColumnRenamed("avg(delay)","avg_delay")
tram_types = tram_types.withColumnRenamed("avg(delay)","avg_delay")
tram_types_hour = tram_types_hour.withColumnRenamed("avg(delay)","avg_delay")

In [ ]:
#transport_avg_delays.write.parquet('/user/%s/final/parquet/transport_avg_delays' % username)
#hour_avg_delays.write.parquet('/user/%s/final/parquet/hour_avg_delays' % username)
train_types = train_types.withColumnRenamed("avg(delay)","avg_delay")
train_types.write.parquet('/user/%s/final/parquet/train_types' % username)
bus_types= bus_types.withColumnRenamed("avg(delay)","avg_delay")
bus_types.write.parquet('/user/%s/final/parquet/bus_types' % username)
tram_types = tram_types.withColumnRenamed("avg(delay)","avg_delay")
tram_types.write.parquet('/user/%s/final/parquet/tram_types' % username)
train_types_hour.withColumnRenamed("avg(delay)","avg_delay").write.parquet('/user/%s/final/parquet/train_types_hour' % username)
bus_types_hour.withColumnRenamed("avg(delay)","avg_delay").write.parquet('/user/%s/final/parquet/bus_types_hour' % username)
tram_types_hour.withColumnRenamed("avg(delay)","avg_delay").write.parquet('/user/%s/final/parquet/tram_types_hour' % username)

In [ ]:
default_delay = df.select(F.mean(F.col('delay')).alias('mean_delay'),).collect()


In [ ]:
default_delay.show()